In [5]:
import pyspark
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
df = pd.DataFrame([2,3,4])
df

,0
0,2
1,3
2,4


In [6]:
sc = pyspark.SparkContext()

#con esto podemos trabajar con los RDD cutres
sc.parallelize([1,2,3]).count()

# si creamos un contexto de SQL (a partir del SparkContext)
# entonces ya podemos usar todas las funcionalidades de los dataframe de SQL
sql = pyspark.SQLContext(sc)

# Leer y escribir: parquet, csv, ...

Para que funcionara la escritura de parquets hubo que añadir una variable de entorno que apuntara a un .exe que se descarga aqui:

https://github.com/steveloughran/winutils/blob/master/hadoop-2.7.1/bin/winutils.exe

se pega en un directorio C:/hadoop/bin/wintutils.exe

se añade HADOOP_HOME que apunte a: C:/hadoop/

RESETEAMos el notebook y ya funciona

YA FUNCIONA

In [19]:
df = spark.read.\
            csv("C:/Users/vr255019/Downloads/annual-enterprise-survey-2019-financial-year-provisional-csv.csv").\
            select('_c0','_c1','_c2','_c3','_c4')
df.show(5,truncate=False)

+----+---------------------------+--------------------+--------------------+------------------+
|_c0 |_c1                        |_c2                 |_c3                 |_c4               |
+----+---------------------------+--------------------+--------------------+------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|Units             |
|2019|Level 1                    |99999               |All industries      |Dollars (millions)|
|2019|Level 1                    |99999               |All industries      |Dollars (millions)|
|2019|Level 1                    |99999               |All industries      |Dollars (millions)|
|2019|Level 1                    |99999               |All industries      |Dollars (millions)|
+----+---------------------------+--------------------+--------------------+------------------+
only showing top 5 rows



In [17]:
df.write.parquet("DATOS/tabla1") # este directorio esta en el mismo lugar que los notebooks. Datos es una subcarpeta y tabla1 el nombre del parquet

In [14]:
df.write.saveAsTable("people5") # escrito asi te lo deja en raiz_notebooks/spark-warehouse con el nombre people5

In [16]:
df.write.saveAsTable("people6") # escrito asi te lo deja en raiz_notebooks/spark-warehouse con el nombre people5

In [17]:
spark.sql("show tables").show(4) # asi podemos ver lo que hay en raiz_notebooks/spark-warehouse

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  people5|      false|
| default|  people6|      false|
+--------+---------+-----------+



# Tech Ant

In [7]:
import yaml
from pathlib import Path

DEFAULT_CONF_ENV = "discovery"

def load_default_tables():

    tables_conf = "C:/Users/vr255019/OneDrive - Teradata/Desktop/new_training/GEAOSP_UC34_Quality_DS/tech_ant/remedies/conf/discovery/tables.json"
    return load_json(tables_conf)

In [8]:
def create_yarn_connection(app_name):
    conf = pyspark.SparkConf()    
    return SparkSession.builder.config(conf=conf).enableHiveSupport().appName(app_name).getOrCreate()

spark = create_yarn_connection('Bau_technical_anticipation')
spark.sql('set spark.sql.hive.convertMetastoreParquet=false')

DataFrame[key: string, value: string]

In [9]:
import json
from pathlib import Path

def load_json(path):

    with Path(path).open("r") as fp:
        result = json.load(fp)
   
    return result

In [18]:
class Pipeline(object):
   
    def __init__(self, spark_session, pipeline_conf):
   
        #self.logger = logging.getLogger(__name__)
        self.spark = spark_session
        self.conf = pipeline_conf if isinstance(pipeline_conf, dict) else 'load_json(pipeline_conf)'

class ModelPipeline(Pipeline):

    def __init__(self, spark_session, pipeline_conf):

        super().__init__(spark_session, pipeline_conf)
#         self.logger = logging.getLogger(__name__)
#         self.model_transformation_handler = self.get_model_transformer()(spark_session=spark_session, pipeline_conf=self.conf["model_transformation"])
#         self.scinet = SciNet(self.conf["scinet"])

class PredictModel(ModelPipeline):
  
    def __init__(self, spark_session, pipeline_conf):

        super().__init__(spark_session, pipeline_conf)
        self.tables = self.conf["tables"] if "tables" in self.conf else load_default_tables()
        
    def drop_table(self, table_name):
        print(table_name)
        return self.spark.sql("DROP TABLE IF EXISTS {table}".format(table=table_name))

    def drop_scoring_hist_table(self):
      
        self.drop_table(table_name=self.tables["scoring_hist_table"])

    def drop_predictions_fts_hist_table(self):
        """Helper function to drop the predictions dataset table set by conf
        """
        self.drop_table(table_name=self.tables["predictions_fts_hist_table"])

predict_pipe = PredictModel(spark_session=spark, pipeline_conf='PREDICT_CONF')
predict_pipe.drop_predictions_fts_hist_table()

people5
